In [3]:
####################### Library for web crawling #########################
import os
import re
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

####################### Library for saving data ##########################
import pandas as pd

####################### Library for Extracting keywords and drawing wordclouds ##########################
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

##################### variables related selenium ##########################
options = webdriver.ChromeOptions()
#options.add_argument('headless') # 크롬이 실행되지 않고 background로 작업할 수 있게끔 함
options.add_argument('lang=ko_KR')
chromedriver_path = "./chromedriver" # user's chromedriver path
driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기
wait = WebDriverWait(driver, 10)

def main():
    global wait
    word_to_find = '아워스코프' # 검색어
    driver.get('https://search.musinsa.com/search/musinsa/goods?q={}&list_kind=small'.format(word_to_find))
    dic = {}
    travel(dic) # 나온 상품들 탐색
    df = pd.DataFrame.from_dict(dic, orient='index').transpose()
    df.to_excel('./result_{}.xlsx'.format(word_to_find), encoding = 'utf8')
    
def travel(dic): # 검색해서 나온 상품들 순차적으로 enter후 크롤링
    global wait
    
    menu = driver.find_element(By.CLASS_NAME, 'n-search-nav')
    products_num = menu.find_element(By.CSS_SELECTOR, 'a:nth-child(3) > span').text # 검색된 상품 개수
    products_num = int(products_num.replace(',', ''))
    page = 1
    for i in range(1, products_num + 1):

        index = i % 90 # 한 페이지에 90개의 상품이 display
        if i != 1 and index == 1: # index가 1이면 page를 넘겨야 한다는 것
            page += 1
            next_page = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#goodsList > div.pagination-box.box > div > div > a:nth-child({})'.format(page + 2))))
            next_page.send_keys(Keys.ENTER) # 다음 페이지로

            if page > 10: page = 1
                
        driver.implicitly_wait(10)
        sleep(3)
        
        # item select
        item = driver.find_element(By.XPATH,'//*[@id="searchList"]/li[{}]'.format(index))
        item = item.find_element(By.CLASS_NAME, 'li_inner')
        item = item.find_element(By.CSS_SELECTOR, 'div')
        
        try: close_popUp() # 여러 가지 push 알림 때문에 탐색에 문제가 생기는 것을 방지
        except: pass
        
        item.click()
        driver.implicitly_wait(10)
        title = driver.find_element(By.CLASS_NAME, 'product_title') # 상품명 
        title = title.find_element(By.TAG_NAME, 'em').text
        print('########## {} ##########\n'.format(title))
        reviews = []
        crawling(reviews) # 리뷰 크롤링
        dic[title] = reviews
        driver.back() # 뒤로가기(상품 리스트가 있던 페이지로)
        driver.implicitly_wait(10)
        
def crawling(reviews): # 리뷰 크롤링 진행 함수  
    wait = WebDriverWait(driver, 10)
    try:
        pop3 = driver.find_element(By.XPATH,'//*[@id="ch-plugin-core"]/div[2]')
        driver.execute_script("arguments[0].style.visibility='hidden'", pop3) # 팝업창 제거
    except: pass
    # 스타일 후기, 상품 사진 후기, 일반 후기 중 일반 후기를 탐색하도록 함, 일반적인 고객의 리뷰를 보기 위함
    normal_review = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#estimateBox > div.tabBox > ul > li:last-child')))
    reviews_lim = normal_review.text.replace(',', '')
    reviews_lim = int(re.findall(r'\d+', reviews_lim)[0])
    rev_index = 1
    normal_review.click()
    sleep(2)
    
    # 리뷰 몇 페이지 있는지 
    try:
        pages = driver.find_element(By.CSS_SELECTOR, '#wrapEstimateList > div > div.nslist_bottom > div.box_page_msg')
        pages = int(pages.text.split(' ')[0])
        reviews.append(reviews_lim)
        rate = driver.find_element(By.XPATH, '//*[@id="estimate_point"]/p/span')
        reviews.append(rate.text)
    except: return
    
    for i in range(1, pages + 1): # 페이지 넘겨가면서 탐색
        print('---------- page No {} ----------'.format(i), '\n')
        page = i % 5
        if page == 0: page = 5
        if (i - 1) % 5 == 0 and i != 1: # 5개의 페이지씩 display 되므로 그 이후에는 넘겨주어야 함
            review_page = driver.find_element(By.CSS_SELECTOR,'#wrapEstimateList \
                > div > div.nslist_bottom > div.pagination.textRight > div > a:nth-last-child(2)').send_keys(Keys.ENTER)
        else:
            # 해당 페이지 진입
            review_page = driver.find_element(By.CSS_SELECTOR,'#wrapEstimateList \
            > div > div.nslist_bottom > div.pagination.textRight > div > a:nth-child({})'.format(page + 2)).send_keys(Keys.ENTER)
            
        driver.implicitly_wait(10)
        sleep(3)
        #wrapEstimateList > div
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        reviews_num = len(soup.select('#wrapEstimateList > div > .review-list')) # 리뷰 총 개수 
        
        print(reviews_num)
        for j in range(1,reviews_num + 1): # 해당 페이지 리뷰 개수만큼
            print(rev_index, '/', reviews_lim)
            if  rev_index > reviews_lim: return
            review = driver.find_element(By.XPATH, '//*[@id="wrapEstimateList"]/div/div[{}]/div[4]/div'.format(j))
            #review = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="wrapEstimateList"]/div/div[{}]/div[4]/div'.format(j))))
            print(review.text,'\n')
            reviews.append(review.text)
            rev_index += 1        
            
def close_popUp(): # 팝업창 제거
    pop1 = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[4]')
    driver.execute_script("arguments[0].style.visibility='hidden'", pop1)
    pop2 = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[5]')
    driver.execute_script("arguments[0].style.visibility='hidden'", pop2)     
    pop3 = driver.find_element(By.XPATH,'//*[@id="ch-plugin-core"]/div[2]')
    driver.execute_script("arguments[0].style.visibility='hidden'", pop3)        
    
if __name__ == "__main__":
    main()

########## Cashmere Blended Mac Coat (Deep Navy) ##########

########## Zip-up Wool Bellows Jacket (Black) ##########

---------- page No 1 ---------- 

10
1 / 15
같은 가격대 다른 브랜드에 비해 마감이 좀 아쉽긴 하지만 핏은 정말 좋습니다 

2 / 15
저한테는 2사이즈도 조금은 작은 느낌이 있어요.. 그래도 너무 이뻐서 만족해요 ㅎㅎ! 

3 / 15
기장감도 적당하고 따뜻할 거 같아요 잘 입고 다니겠습니다 

4 / 15
적당한 두께에 숏한 기장감이어서 레이어드 해서 입으면 이쁠꺼 같아요
팔 기장이 조금 긴듯한데 단추나 똑딱이 단추로 조절 가능했으면 더 좋았을꺼 같아요 

5 / 15
옷은 이쁜거 같은데 저랑은 좀 뭔가 안어울리네요 ㅠㅠ 

6 / 15
사이즈감이 엄청 작게 나와서 평균남자 스펙이시면 무조건 2사이즈 가시는게 맞네요 

7 / 15
아 역시 이쁩니다 진짜 잘산거같아요 거의 코트두께정도로 탄탄해서 따뜻하게 입기 좋네요 

8 / 15
생각보다 두껍고 이쁜데 팔쪽이 깔끔하게 떨어지지않으며 카라가 이상하게 다림질?되어왓네요 

9 / 15
1사이즈도 조금 품이 큰 느낌이네요! 그치만 디자인은 넘 이쁩니다 

10 / 15
사이즈는 딱 맞는 감이 있어용 껴입고는 못입을듯ㅠㅠ 저랑 체형 비슷하신분들 참고
보온성은 간절기용 

---------- page No 2 ---------- 

5
11 / 15
두께감이랑 기장이 너무 마음에 들어서 만족합니다 

12 / 15
짧게 나와서 레이어드로 엣지 있게 스타일하면 너무 이쁠 것 같지만, 가을 옷 정도로 생각이 되네요..
투웨이 지퍼가 좋아서 두번째로 구매하는 아워스코프 자켓이지만 겨울에 투웨이는 큰 의미는 없을 것 같아요..(추우니까)
그리고 사이즈가 세세미오버핏이예요.
짧고 팔은 길고 어깨서는 살짝 드롭으로 떨어지고
큰 문제가 있다면 위에 스텝 옷 사진을 보면 알 수 있다

########## Tasmania Wool Rib Knit (Sky Blue) ##########

########## Hidden Button Blazer (Light Olive) ##########

---------- page No 1 ---------- 

6
1 / 6
색감이 화면보다 이뻐서 좋네요
사이즈도 좀 오버핏이라서 좋아요 

2 / 6
구매했는데 너무 만족하면서 잘 입고있습니다 추천합니다!! 

3 / 6
좋은거 같아요 느낌 있네요 ㅎㅎ 잘입겟습니다 아주 이뻐요 

4 / 6
원단은 부들부들한게 좋지만 그만큼 탄탄하진 못합니다. 초록빛은 많이 돌지않고 카키브라운 색상으로 크게 부담스럽진 않습니다. 

5 / 6
색상도 이쁘고 재질도 좋고 두께감도 적당해서 좋고 정핏 입니다 

6 / 6
재질이 약간특이하네요 핏좋고 색감좋습니다 오래는 못입을거같네요 

########## Cotton Bellows Jacket (Deep Navy) ##########

---------- page No 1 ---------- 

6
1 / 6
1사이즈 좋습니다. 자칫하면 공장 잠바지만 그 느낌도 또 나쁘지 않은 것 같기도..? 

2 / 6
색감도 핏도 두께도 너무 만족스러운데, 다만 살짝 품이 딱 맞네요! 왜 이제야 샀는지 후회되지만 그래도 할인으로 저렴하게 건지니 간절기 자켓 찾으시는 분들한텐 강추합니다! 

3 / 6
너무 이뻐요! 핏이 너무 트렌드합니다 감사합니다! 

4 / 6
저는 체형이 좀 있어서 아재핏이 됐지만 마르신 분들은 잘 소화하실 것 같습니다 

5 / 6
아워스코프 이번 SS시즌 데님과 니트를 구매해봤더니
마음에 들어서 아우터도 구매해봤습니다
전체적인 핏과 사이즈는 마음에 듭니다
그런데 혹시 급하게 리오더를 하신건지
옷에 실밥들이 하나도 정리 안되어있고
실이 뭉쳐서 미싱된곳도 꽤 있네요
30분정도 실밥 제거하니 현타옵니다 

6 / 6
색감 괜찮고 길이도 적당함 근데 밑부분에 고무줄이라고 해야되나?? 그부분이 생각보다 많이 쪼그라듬 


########## Strap Blazer Jacket (Navy) ##########

---------- page No 1 ---------- 

3
1 / 3
색도 예쁘고 디자인이 독특해서 눈길이 갔어요 일상 코디로도 괜찮을 것 같아요 

2 / 3
겉감 안감 부들부들하니 너무 좋구요 핏도 잘 떨어집니다 굳굳 

3 / 3
너무 너무 만족합니다!!! 봄 초여름 입고다닐 수 있을거 같아요 

########## Melton Double Coat (Charcoal) ##########

---------- page No 1 ---------- 

1
1 / 1
가격 값은 한다고 생각합니다.
사이즈도 적당하고요
추천드립니다. 

########## Wrinkle Basic Shirts (White) ##########

---------- page No 1 ---------- 

1
1 / 1
사이즈도 정 핏 나오고 이쁩니다 시워한고 한여름에 입어도 될꺼같아요 

########## Elbow Patch Sweat Shirts (Mocha Beige) ##########

########## Two Paneled Long Sleeve (Brown) ##########

---------- page No 1 ---------- 

4
1 / 4
아니 색감이 진짜로 너무 예술이에요 꼭사시길바랍니다,, 

2 / 4
가운데 절개선 포인트 좋네요
생각보다 정핏이라 아쉽습니다 

3 / 4
생각보다 두껍고 평범할수 있는 롱슬리브인데 가운데 포인트를줘서 좀 괜찮은것 같습니다 

4 / 4
반팔티랑 맨투맨의 중간정도?
지금날씨에 입기 좋을 것 같아요~!
만족합니다
체형보정 약간 되는 것 같아요 

########## Colorful Basic T-Shirts (Light Gray) ##########

########## Spectrum Sweat Shirts (Orange) ##########

---------- page No 1 ---------- 

1
1 / 1


In [5]:
####################### Library for web crawling #########################
import os
import re
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

####################### Library for saving data ##########################
import pandas as pd

####################### Library for Extracting keywords and drawing wordclouds ##########################
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

##################### variables related selenium ##########################
options = webdriver.ChromeOptions()
#options.add_argument('headless') # 크롬이 실행되지 않고 background로 작업할 수 있게끔 함
options.add_argument('lang=ko_KR')
chromedriver_path = "./chromedriver" # user's chromedriver path
driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기
wait = WebDriverWait(driver, 10)

def crawling(reviews): # 리뷰 크롤링 진행 함수  
    wait = WebDriverWait(driver, 10)
    try:
        pop3 = driver.find_element(By.XPATH,'//*[@id="ch-plugin-core"]/div[2]')
        driver.execute_script("arguments[0].style.visibility='hidden'", pop3) # 팝업창 제거
    except: pass
    # 스타일 후기, 상품 사진 후기, 일반 후기 중 일반 후기를 탐색하도록 함, 일반적인 고객의 리뷰를 보기 위함
    normal_review = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#estimateBox > div.tabBox > ul > li:last-child')))
    reviews_lim = normal_review.text.replace(',', '')
    reviews_lim = int(re.findall(r'\d+', reviews_lim)[0])
    rev_index = 1
    normal_review.click()
    sleep(2)
    
    # 리뷰 몇 페이지 있는지 
    try:
        pages = driver.find_element(By.CSS_SELECTOR, '#wrapEstimateList > div > div.nslist_bottom > div.box_page_msg')
        pages = pages.text.split(' ')[0]
        pages = int(pages.replace(',',''))
        reviews.append(reviews_lim)
        rate = driver.find_element(By.XPATH, '//*[@id="estimate_point"]/p/span')
        reviews.append(rate.text)
    except: 
        print(1)
        return
    
    for i in range(1, pages + 1): # 페이지 넘겨가면서 탐색
        print('---------- page No {} ----------'.format(i), '\n')
        page = i % 5
        if page == 0: page = 5
        if (i - 1) % 5 == 0 and i != 1: # 5개의 페이지씩 display 되므로 그 이후에는 넘겨주어야 함
            review_page = driver.find_element(By.CSS_SELECTOR,'#wrapEstimateList \
                > div > div.nslist_bottom > div.pagination.textRight > div > a:nth-last-child(2)').send_keys(Keys.ENTER)
        else:
            # 해당 페이지 진입
            review_page = driver.find_element(By.CSS_SELECTOR,'#wrapEstimateList \
            > div > div.nslist_bottom > div.pagination.textRight > div > a:nth-child({})'.format(page + 2)).send_keys(Keys.ENTER)
            
        driver.implicitly_wait(10)
        sleep(3)
        #wrapEstimateList > div
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        reviews_num = len(soup.select('#wrapEstimateList > div > .review-list')) # 리뷰 총 개수 
        
        print(reviews_num)
        for j in range(1,reviews_num + 1): # 해당 페이지 리뷰 개수만큼
            print(rev_index, '/', reviews_lim)
            if  rev_index > reviews_lim: return
            review = driver.find_element(By.XPATH, '//*[@id="wrapEstimateList"]/div/div[{}]/div[4]/div'.format(j))
            #review = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="wrapEstimateList"]/div/div[{}]/div[4]/div'.format(j))))
            print(review.text,'\n')
            reviews.append(review.text)
            rev_index += 1  

driver.get('https://store.musinsa.com/app/goods/1139338') # 한 상품 페이지 주소
driver.implicitly_wait(10)
title = '엘무드 화란 세미오버 가디건 클라우드 그레이'
dic = {}
reviews = []
crawling(reviews)
dic[title] = reviews
df = pd.DataFrame.from_dict(dic, orient='index').transpose()
df.to_excel('./result_{}.xlsx'.format(title), encoding = 'utf8')

---------- page No 1 ---------- 

10
1 / 10910
상체가 긴편이라 좀 짧지만 그거빼고는 다 좋아요 참고하셔요 

2 / 10910
제가 가디건을 처음 사보는데 정말 완전 이쁘네요 

3 / 10910
코디하기 싶고 핏 좋아요. 따뜻하고 가벼워 만족합니다 

4 / 10910
기본 가디건으로 제격이에요 하나 필요해서 샀는데 아주 만족스러워요 

5 / 10910
진짜 너무 이뻐서 계속 손이 가네요. 다른 색상도 사고싶어요 

6 / 10910
선물 주려고 샀는데 깔끔하고 이뻐요
굿굿이에요 

7 / 10910
니트류는 엘무드만 입는듯 보들보들 핏도 좋고 최고 

8 / 10910
색상은 보이는 그대로입니다. 세미 오버핏으로 과하지 않는 핏이에요! 재질은 엄청 부드럽지는 않지만 거슬릴 정도는 아닌 것 같아요. 가격 대비 만족합니다~ 

9 / 10910
배송도 비교적 빠르고 제품 디자인도 예상했던것처럼 무난합니다. 

10 / 10910
촉감이 너무 부드럽고 핏도 이쁘게 떨어져서 좋아요 

---------- page No 2 ---------- 

10
11 / 10910
너어어무 예뻐요 역시 갓무드인가요
핏감 재질 완벽해요 

12 / 10910
역시 니트 가디건 맛집 인거 같네요 핏감이랑 너무 좋고요 아쉬운점이 있다면 소매가 좀 긴거같아요.. 원래 근데 조금 덮는 느낌으로 입는거 같긴한데 저한테는 이런 감성이 좀 별로 인거 같네요;; ㅠㅠ 그래도 잘입고 다니겠습니다!! 

13 / 10910
약간 오버하면서도 몸에 맞게 사이즈 이쁘게 나오고 색감이랑 재질도 좋아서 아주 만족합니다 

14 / 10910
색감 좋고 질도 좋아서 자주 입을 거 같아요! 강추합니다. 

15 / 10910
부드럽고 따뜻해서 반팔 하나 입고 위에 걸쳐입기 좋습니다 ! 핏도 좋아요 ~ 

16 / 10910
이 가격에 이만한 가디건 찾기힘듭니다
얼른구입하세요 

17 / 10910
배송이 좀 오래걸렸습니다. 예약배송인 것을 구매하고 알았어요. 옷에 대

10
151 / 10910
너무 두껍지도 너무 얇지도 않은 그런 가디건이에요 

152 / 10910
이쁩니다! 재질도 맘에 들고 핏도 좋아요. 하나 단점이 있다면 주변에 입는사람이 꼭 한두명 보인다는거 .. 

153 / 10910
운동 조금 한 몸인데 세미오버정도로 잘 맞네요 퀄리티도 가격대비 좋은것 같습니다! 

154 / 10910
디자인은 그냥 평범한거 같은데 재질이나 봉제, 마감이 좋아요 

155 / 10910
배송까지 좀 오래 걸렸지만 만족하면서 입고있습니다 

156 / 10910
저한테는 세미오버였지만 정핏으로 떨어졌습니다! 예쁘고 좋아요!! 

157 / 10910
배송이 너무 늦었어요ㅠㅠ 근데 무게감도 좋고 질도 좋아요 용서합니당 

158 / 10910
핏 이쁘고 사이즈도 오버하게 잘 맞아요,, 다른색도 살게요 

159 / 10910
좀 많이 짧고 재질은 너무 바람막이 등산복 느낌이에여 

160 / 10910
촉감이 부드러워서 좋아요.
한사이즈 큰거 샀으면 더 좋을거 같긴하네요. 

---------- page No 17 ---------- 

10
161 / 10910
그냥 리뷰만 보고 샀는데 싼 값에 너무 잘샀네요 가성비 최고에요 ! 

162 / 10910
정말 만족하면서 입고 있습니다. 46사이즈는 저한테 살짝 작은감은 있어요. 

163 / 10910
검은색 가디건은 기본 필템이죠
촉감 색감 사이즈 다 좋습니다^^ 

164 / 10910
재질이 넘 부드럽고 고급스럽습니다 핏도 넘 이쁘게 나오네여 

165 / 10910
피부톤이 밝으신분들이 입으면 더 예쁘게 잘 어울릴것 같아요 

166 / 10910
좀 오래기다려서 받긴했지만...그래도 색깔도 이쁘고 자체도 부드러워서 좋네요! 잘 입고 다닐거 같아요:) 

167 / 10910
좋습니다! 보풀만 안일어나면 더할나위 없겠네요 

168 / 10910
제품 자체는 맘에 들었으나 배송 부분에서는 실망이에요 ㅠㅠ 10.25 예약배송이였으나... 26일에 배송이 시작되는... 쨋

부드럽고 재질좋아요 근데 여자가 입기에 오버핏이라해도 어깨선ㅇㅣ 안이쁘게 내려옴 

---------- page No 31 ---------- 

10
301 / 10910
제 기준에 딱 가을에 아우터로도 입기 좋고 겨울에는 이너류 입기 좋은 핏입니다! 

302 / 10910
배송이 늦긴했지만 색이랑 핏 둘다 완전 만족이네요 

303 / 10910
사이즈 너무 딱맞고 이쁘네요 벌써 두개째 같은 걸로 색만 달리해서 샀네요~~ 

304 / 10910
사이즈는 딱 맞고 디자인도 색상도 다 마음에 드네요 

305 / 10910
생각보다 도톰해서 겨울에도 입기 좋을것 같아요. 

306 / 10910
배송은 느렸지만 옷은 만족해서 좋습니다
엘무드가 가디건 이쁘긴하네요 

307 / 10910
배송이 좀 오래걸렸는데 상품은 대만족합니다.
잘 입겠습니다 

308 / 10910
사이즈는 딱인데 단추를 풀어보니 눌림자국이 심함....
한번 다려서 입어야함 

309 / 10910
옷 너뮤 이쁘고 사이즈도 잘맞아서 자주 입을거 같아요 

310 / 10910
착용감이 진짜 좋네요
아주 부드럽고 몸에 착 감겨요 

---------- page No 32 ---------- 

10
311 / 10910
와.. 제가 가디건을 이번에 처음 구매한건데 진짜 너무 마음에 들어요 엄청 부드럽고 두께도 적당히 두껍고 진짜 새로운 경험을 하는 느낌이네요 사이즈도 딱 좋고 입어보니 사람이 분위기가 달리보이는 듯한 느낌입니다 잘 입을게요..!! 

312 / 10910
옷의 재질도 너무 좋고 가을 날씨에 입으면 너무 좋을거 같아요! 

313 / 10910
너무 맘에 듭니다 역시 믿고 샀어요 담에또살게요 

314 / 10910
깔금하고 이쁘게 입을때 좋을거 같아요 아우터 이너로도 이쁠거 같아여 

315 / 10910
유튜브 리뷰보고 삿는데
리뷰할 만 하네여
이쁘게 잘 입을 게여 

316 / 10910
퍼펙트 이 세글자로 표현할 수 있겠네요호호
20자 

317 / 10910
생각보

10
451 / 10910
사이즈 세미오버핏이라이쁘고 부들부들해서 좋아요 

452 / 10910
가을에 입기좋고 디자인 마음에 듭니다 단추도 이뻐요 

453 / 10910
음 고민중이면 일단 사세요
확실히 후회는 안하실거에요ㅋ 

454 / 10910
46 / 48 둘 중 엄청 고민했는데 저스트하게 입으려고 46 샀습니다 ㅎㅎ 그리고 정말 만족 중이에요 ~ 왜 이렇게 엘무드 가디건이 인기가 많은지 궁금했는데 바로 수긍을 해버렸네요 :) 색감도 일반적인 그레이톤에서 한 단계 밝은 느낌이고 재질도 너무 부드럽고 ,, 만능입니다 ㅎㅎ 

455 / 10910
일상에서 이뻐요 집앞에 카공할때 자주 입고 다녀요 

456 / 10910
이쁘고 촉감도 괜찮은데 완전 회색이 아닙니다 참고 사야합니다. 

457 / 10910
ㅋㅋ진짜 잘샀다 유튜브 리뷰보고 구매했는데 이번 무신 쇼핑에서 제일 잘샀다고 생각합니다ㅎㅎ 

458 / 10910
소매가 조금 길기는 한데 크게 문제 될 것은 없어요 

459 / 10910
이쁘게 잘 입고 있습니다 정말 좋아요 최고! 재구매 의사 있습니다 

460 / 10910
옷이쁘고 입기편해요 집앞산책나갈때나 마실갈때 좋네요 

---------- page No 47 ---------- 

10
461 / 10910
깡스타일리스트님 보고 구매했는데
갑자기 추워진 요즘 가볍게 레이어드해서 입기도하고 오후에 더울땐 단독으로 가디건만 입어도 이쁘네요 ㅎㅎ 

462 / 10910
생각보다 살짝 밝은 정도, 오버핏 원하시면 한치수 업하세요 

463 / 10910
옷핏이 잘맞게 이쁩니다.
어깨도넓어보이고 재질도부들좋습니다. 

464 / 10910
가성비 좋고 깡스타보고 샀어요~!
너무좋아용 잘 입을게요 

465 / 10910
생각보다 너무 만족합니다 사진 그대로라 너무 만족해요 

466 / 10910
짱짤하고 딱 제가 원했던 가디건입니다 세미오버라 널널해용 

467 / 10910
품질은 좋구요 정핏도 오버핏도 아닌만큼 그걸 잘 이용하면 좋

---------- page No 61 ---------- 

10
601 / 10910
엘무드 화란 클라우드 그레이 가디건
46 사이즈 딱 좋습니다 

602 / 10910
입으면 따뜻함 .. 술마시고 침대에 잘못 누웠다가 보풀좀 일어남 ,,ㅠㅠ 

603 / 10910
사진처럼 이쁘긴 한데 몇번 입으니까 보풀이 올라오네요.. 

604 / 10910
50사이즈가 없어서 고민하시는 분 계실텐데 기장은 183cm 기준 골반 정도 높이로 적당합니다. 저 가슴어깨도 작은편 아닌데 느낌 아주 괜찮아요. 

605 / 10910
사이즈도 딱 맞고 너무 편하고 좋습니다 !
잘 입을게요! 

606 / 10910
옷핏이 진짜 너무 이쁘게 잘 떨어집니다,,,색감도 굳 

607 / 10910
색감 사이즈 다 좋고 부드러워서 맨 몸에도 입어도 되네요 크롭 가디건보다 부드러움 

608 / 10910
적당한 두께감이 있어서 쌀쌀한 날씨에도 입기 좋은 것 같습니다! 

609 / 10910
엘무드 화란 세미오버 가디건 클라우드 그레이 좋아요 

610 / 10910
부들부들하고 촉감좋고 어깨넓어보이는 효과있어요 

---------- page No 62 ---------- 

10
611 / 10910
색감도 좋고 핏도 잘 나옵니다 블랙가디건은 하나쯤 있어야 할 것 같아요 

612 / 10910
진짜 남친룩 찰떡입니다 많이 파세요~~~~ 너무 잘 어울려요 

613 / 10910
어깨쪽이 넉넉해서 적당한 핏감으로 입기 좋아요. 

614 / 10910
전에 검정색도 샀고 마음에 들어서 색만 바꿔 구매했어요 

615 / 10910
생각보다 더 이쁜거 같습니다. 다음에 또 구매 할깨여 

616 / 10910
진짜 이뻐용 추천드립니다 포장도 고급스럽게와서 만족만족 

617 / 10910
색감도 좋고 부드러워서 입기 편하네요. 여러 색이랑도 잘어울려서 만능템입니다 

618 / 10910
재질이나 색감, 입었을때 감촉 등 모든것이 만족됨 

619 / 10910
코디하기 쉽고 

...

10
10361 / 10910
부드럽고 색감도 되게 예뻐요! 털빠지는게 좀 있긴해요 

10362 / 10910
174 66 무조건 m 가세요 저그트입니다 딱 좋아용 

10363 / 10910
엘무드 니트도 사고 가디건도 삿는데 역시좋네요
깔끔하니 미니멀에 제격입니다! 

10364 / 10910
핏도 정말이쁘고 재질도좋고 모두만족합니다 ㅎㅎ 재구매의사있음 

10365 / 10910
두께도 적당하고 생각보다 많이 부드럽네요 좋은거같아요 

10366 / 10910
생각보다 어깨가 많이 오버하게 나왔지만 핏은 예쁘게 떨어지네요 엘무드 이름값하는 제품입니다 ㅎㅎ 

10367 / 10910
어깨가 많이 크게나왔어요 드롭숄더 좋아하시는분한태 추천드려요 

10368 / 10910
어깨가 많이 크게나왔어요 드롭숄더 좋아하시는분한태 추천드려요 

10369 / 10910
눈여겨 보고있다가 블프떄샀는데 포장도 고급스럽게 오고 부자재도 잘어울리고 팔쪽 떨어지는게 자연스럽게 드롭되서 이쁩니다 2개 구매했어요
178/70kg 사이즈 딱좋아욤 

10370 / 10910
무난하면서 입기 딱 좋네요
담에 또 여기서 살게요~ 

---------- page No 1038 ---------- 

10
10371 / 10910
무난하고 좋습니다. 봄, 가을에 잘 입고 겨울에도 이너로 입을듯 

10372 / 10910
색깔 이쁘고 맘에듭니다 살짝큰거같았는데 46은 작앗네요
184/73 

10373 / 10910
친구 생일선물로 샀는데 친구가 좋아해요 170/61 

10374 / 10910
역시 평도 좋고 가격도 비싼 만큼의 질을 보여주는 것 같아요 

10375 / 10910
검정색사고 맘에들어 하나더샀는데 실망안시키네요 

10376 / 10910
가디건 이쁜게 너무 많아서 고민이 진짜 많이 됐는데 후회없는 선택입니다! 옷 너무 이쁘고 원단도 맘에 드네용 아껴입고있어요.. 

10377 / 10910
디자인과핏은아주마음에듭니다 감사합니다 다음에도또시키겟습니다 

10378 /

10490 / 10910
진짜 엘무드는 제가 제일 좋아하는 브랜드인데 역시 너무 부드럽고 핏이 잘 떨어지네요 너무 좋습니다 

---------- page No 1050 ---------- 

10
10491 / 10910
너무너무 예쁩니다... 진짜 저는 가디건이 제 체형에 맞지 않는 옷이라고 생각했지만 이 옷이 그 생각을 바꿔줬습니다.. 체형보완 오집니다. 다들 이거 그냥 사세요. 

10492 / 10910
허허허 좋습니다. 잘입고있습니다. 따듯합니다. 부르르르릉 

10493 / 10910
엘무드는 다른 사람들의 후기도 많고 평도 좋아서 구매할까 고민하고 있었는데 이번에 타임세일이 나와서 구매했습니다. 디자인과 색감이 너무 이쁘고 세미오버핏으로 나와서 좋지만 안에 보풀이 많이 묻어나오는게 조금 아쉽네요.ㅠ 

10494 / 10910
사이즈 딱 맞고 재질도 좋고 사길 잘했습니다 만족 

10495 / 10910
화란니트 한케이스로는 평가하기 어려워 구매했습니다. 일단 생각보다 괜찮네요 

10496 / 10910
두께도 두툼하고, 촉감도 부드럽다
근데 털이 많이 빠진다.
돌돌이로 몇번 돌려서 입고 다녀야하고
보풀도 걱정된다. 

10497 / 10910
s사이즈 딱 세미오버핏나와요. 부드럽고 이쁜데 보풀걱정이 좀되네요. 

10498 / 10910
생각햇던것보다 더 만족입니다. 좀 클까봐 걱정햇는데 평소 오버핏을 좋아하기 때문에
너무 만족합니다. 

10499 / 10910
저는 5만원대에 구매해서 엄청나게 만족합니다 

10500 / 10910
182 74인데 세미오버하게 이쁘게 떨어집니담. 

---------- page No 1051 ---------- 

10
10501 / 10910
진짜 배송상태부터 완전 만족해서 기분좋게 포장뜯고 입어봣는데 넘나 좋음 진짜 

10502 / 10910
보자마자 고급스럽다는 느낌 받았습니다 가격값하는거같아요 

10503 / 10910
저한테는 기장이 조금길지만 재질이나 핏 다좋아요 

10504 / 10910

두개 주문시켰는데 사이즈도 잘맞고 배송도 바로 다음날와서 좋아요 

10616 / 10910
정핏으로 입으려고 44로 했는데 생각했던 핏 나와서 좋습니다 

10617 / 10910
세일할때 쿠폰먹여서 샀는데 만족합니다. 기본 미니멀하게 입기좋아요. 

10618 / 10910
떨어지는 느낌이랑 컬러랑 너무 예쁘게 나왔어요
사서 후회할 제품이 아닙니다! 

10619 / 10910
이쁜데요 보풀이 생각보다 금방 생기는 느낌?
기대가 커서 그런지 그냥 보통이에여..
그래도 미니멀룩엔 좋은 옷입니다 

10620 / 10910
편한핏으로 잘떨어져서 검정색도 추가 구매했어요 

---------- page No 1063 ---------- 

10
10621 / 10910
역시 엘무드 보들보들하고 생각보다 엄청 따듯하료 가격대비 굳굳굳 

10622 / 10910
너무 이뻐요
보들보들하고 174/70 핏깔끔하게 떨어집니다
가디건은 잘 안입었는데 이건 이뻐서 자주입을거같아요 

10623 / 10910
엘무드 카디건 그레이에 이어서 카멜도 샀는데
가을에 맞는 색이라 이뽀요 

10624 / 10910
부드럽고 두께감도 적당히 있어서 요즘 입기 딱인듯 

10625 / 10910
저렴한 제품이랑 고민 많이했는데 이 제품으로 사길 잘 한 것 같아요. 미니멀하고 단추도 예뻐서 맘에듭니다!
186/60 48사이즈 구매했어요. 

10626 / 10910
좀 더 저렴한 거 살까 고민하다 엘무드 좋아해서 이거로 샀는데 후회 안 합니다.
반팔에 입어도 부드럽고 요즘 날씨에 딱 좋은 고 같아서 좋습니다. 

10627 / 10910
46이랑.48이랑 고민하다가 46갔는데 기장이 생각보다 작은것빼곤 좋습니다 

10628 / 10910
동생 선물했는데 색감이 너무좋아요 핏도 이쁘게 떨어지고 좋네요 

10629 / 10910
고민하다가 샀는데 너무 좋습니다. 배송도 빠르고 사이즈도 딱입니다 ㅎㅎ 

10630 / 10910
솔직히 도메스틱 가디건이 거기서 거기라 생각했는데.. 정말 

10
10741 / 10910
흰색 티에 검은색 털이 살짝 묻지만 이뻐서 좋아요~ 

10742 / 10910
핏 뿐만 아니라 입었을때 원단 느낌이 보들보들합니다. 강추드립니다 

10743 / 10910
재질도 부들부들 너무 좋고 기본템이라 코디하기도 쉽습니다. 은근 가디건 핏 많이 타는데 이건 핏도 좋네요 굿굿 

10744 / 10910
너무 예쁘네요 !! 딱 제가 원하는 재질과 두께와 핏감이엿어요 인기 많은데는 이유가 있네요 클라우드 그레이 색상도 구매예정입니다 ㅎㅎ 170.65 44사이즈 가니까 딱이네요 

10745 / 10910
재질도 좋고 특히 색감이 예쁜데 마감이 좀 아쉽네요 

10746 / 10910
색도 예쁘고 길이도 예쁘고 원단 느낌도 아주 좋습니다 

10747 / 10910
세미 오버하게 좋습니다 두께감 살짝 있고요 지금 입기 딱 좋은거 같아요 입기전에 드라이크리닝 한번하세요 많이 묻어 나오네요 

10748 / 10910
세미 오버하게 좋습니다 두께감 살짝 있고요 지금 입기 딱 좋은거 같아요 

10749 / 10910
엘무드 처음 사용해보는데 다음엔 니트도 사봐야겠네요 ㅎㅎ 재질너무좋고 마감도 다 좋습니다 근데 털빠짐이 좀 있어서 문의 해봐야겠네요 ㅎㅎ 

10750 / 10910
엘무드로 가디건 입문하네요. 품질도 좋고 적당한 세미오버핏이예요.
색감도 좋고 착용했을 때 포근하고 부드럽네요. 

---------- page No 1076 ---------- 

10
10751 / 10910
베이지 구매후 또구매 하는건데 역시 재질좋고 핏좋습니다 

10752 / 10910
색감도 너무 맘에 들고 재질도 부들부들해서 입었을 때 저절로 기분이 좋아지는 가디건이에요. 소매가 살짝길어서 감안하셔야 할 것 같아요 

10753 / 10910
핏이 제가 원하는 세미오버핏이고 되게 보들보들합니다.
잘 산 것 같습니다~~ 

10754 / 10910
색감 이쁘고 부들부들하고 도톰해요 이뻐요..엘무드 굳 

10755 / 10910
187/77이

10864 / 10910
174/68인데 세미 오버핏의 핏감도 매우좋고 색감도 좋고 촉감도 부드러워 아주 좋습니다! 

10865 / 10910
어디든 잘어울려요!! 교복셔츠랑 딱 맞아서 좋아요 

10866 / 10910
살짝 큰데 오버핏으로 입으려구요
손목시보리 거의 없습니다 

10867 / 10910
사이즈도 딱 적당하고 마음에 드네요 나중에 또 구매 할 예정입니다 

10868 / 10910
색감 미쳤구요, 핏 미쳤구요. 거적때기랑 같이 입어도 예뻐요. 

10869 / 10910
재질 핏 너무 마음에 듭니다
세미오버로 잘 맞아요 

10870 / 10910
가을과 잘어울리는 컬러 차콜 구입했어요
실측대비 기장이나 폼 생각보다 여유있어서
잘입습니다.
185 86기준 세미오버핏나옵니다

탄탄하면서 촉감은 부드러워서 맘에들구요
다만 보풀에 약간 약해보이는게 보이는데 아직 착용전이라 모르겠어요

타색상도 구입하고싶을정도로 만족 ^^! 

---------- page No 1088 ---------- 

10
10871 / 10910
가디건은 핏이마음에 안들어 인터넷주문후 항상 후회하고 환불햇는데 처음으로 구매확정햇습니가
좋아여 ㅎㅎ 

10872 / 10910
블랙에이어서 클라우드 그레이도 샀는데 색감이 잘 빠진거 같네요 

10873 / 10910
쿠폰까지해서 6만원에 구매했는데

가성비 매우 좋아요

기본디자인의 가디건이라

유행없이 오래 입기 좋아보입니다

검은 바지에 원단이 날리는건 조금 아쉽지만

심한건 아니라 드라이클리닝 한번 하면

될것 같아요 

10874 / 10910
핏이 너무 좋고 보들보들 해요 사이즈도 딱 좋아요 

10875 / 10910
안그래도 하나 업할려했는데 품절나서 못했내요 178.71 

10876 / 10910
사실 조금 긴가민가했엇는데.. 근데 막상 와서 보니 엄청 좋네요!! 이가격에 이정도면 상당합니다! 

10877 / 10910
살짝 오버하느낌과 옷질감 그리고 색상이 아주 마음에듭니다 

10878 / 10910
